In [1]:
from cellcalc import get_primitive_hkl, MID, get_pri_vec_inplane
from interface_generator import core, print_near_axis, convert_vector_index
from csl_generator import print_list, getsigmas, get_theta_m_n_list
from numpy import array
from numpy import array, dot, round
from numpy.linalg import inv, norm
import numpy as np
import glob
import shutil
import os
from cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, MID
from interface_generator import core, print_near_axis, convert_vector_index, write_trans_file
from numpy import array, dot, round, cross, ceil
from numpy.linalg import inv, det, norm
from numpy import cross, dot, ceil
from numpy.linalg import norm, inv

import numpy as np
import pandas as pd
import shutil
import os
import math
import glob
from scipy.spatial import distance
import matplotlib.pyplot as plt
from numpy.linalg import inv, norm
from numpy import dot

from Element import Element

In [95]:
print_list([1,1,0],100)

Sigma:     1  Theta:   0.00 
Sigma:     3  Theta:  70.53 
Sigma:     9  Theta:  38.94 
Sigma:    11  Theta:  50.48 
Sigma:    17  Theta:  86.63 
Sigma:    19  Theta:  26.53 
Sigma:    27  Theta:  31.59 
Sigma:    33  Theta:  20.05 
Sigma:    41  Theta:  55.88 
Sigma:    43  Theta:  80.63 
Sigma:    51  Theta:  16.10 
Sigma:    57  Theta:  44.00 
Sigma:    59  Theta:  45.98 
Sigma:    67  Theta:  62.44 
Sigma:    73  Theta:  13.44 
Sigma:    81  Theta:  77.88 
Sigma:    83  Theta:  17.86 
Sigma:    89  Theta:  34.89 
Sigma:    97  Theta:  61.02 
Sigma:    99  Theta:  11.54 


In [3]:
def get_a_b(CSL, axis):
    hkl_perp_axis = MID(CSL, axis)
    a, b = get_pri_vec_inplane(hkl_perp_axis, CSL).T

    if(norm(cross(axis,[1,0,0])) < 1e-8):
        b = a + b
    elif (norm(cross(axis,[1,1,1])) < 1e-8):
        if dot(a,b) < 0:
            b = a + b
        b = a + b
    if (abs(norm(a) - norm(b)) < 1e-8):
        raise RuntimeError ('the tow vectors are identical!')

    return a.T, b.T

In [4]:
def get_STGB_MLs(CSL, n_1, n_2):
    hkl_1 = MID(CSL, n_1)
    hkl_2 = MID(CSL, n_2)

    return hkl_1, hkl_2

In [269]:
my_interface = core('Cu_mp-30_conventional_standard.cif','Cu_mp-30_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


In [13]:
np.array(np.round(dot(inv(my_interface.lattice_1), my_interface.lattice_1),8),dtype = int)

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [17]:
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [20]:
np.dot(my_interface.lattice_1,np.array([1,1,1]).T).T

array([3.621262, 3.621262, 3.621262])

In [270]:
axis = np.array([1,1,0])

In [271]:
# ここではconventional cellをmy_interface.lattice_1にして計算をしている
# axis = np.array([1,1,1])
a,b = get_pri_vec_inplane(axis, np.array(np.round(dot(inv(my_interface.lattice_1), my_interface.lattice_1),8),dtype = int)).T
cell = np.column_stack((axis,a,b))
my_interface.atoms_1, my_interface.elements_1,my_interface.lattice_1 = super_cell(cell,np.eye(3),my_interface.atoms_1,my_interface.elements_1)
my_interface.atoms_2, my_interface.elements_2,my_interface.lattice_2 = my_interface.atoms_1.copy(), my_interface.elements_1.copy(),my_interface.lattice_1.copy()
my_interface.lattice_2_TD = my_interface.lattice_1.copy()

In [4]:
a

array([ 0., -1.,  1.])

In [5]:
b

array([-1.,  0.,  1.])

In [6]:
cell = np.column_stack((np.array([1,1,1]),a,b))

In [7]:
cell

array([[ 1.,  0., -1.],
       [ 1., -1.,  0.],
       [ 1.,  1.,  1.]])

In [9]:
from interface_generator import super_cell

In [56]:
my_interface.atoms_1

array([[0. , 0. , 0. ],
       [0. , 0.5, 0.5],
       [0.5, 0. , 0.5],
       [0.5, 0.5, 0. ]])

In [67]:
super_cell(cell,np.eye(3),my_interface.atoms_1,my_interface.elements_1)

(array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 3.33333333e-01,  3.33333333e-01,  3.33333333e-01],
        [ 6.66666667e-01,  6.66666667e-01,  6.66666667e-01],
        [ 0.00000000e+00,  5.00000000e-01,  0.00000000e+00],
        [ 3.33333333e-01,  8.33333333e-01,  3.33333333e-01],
        [ 6.66666667e-01,  1.66666667e-01,  6.66666667e-01],
        [-2.77555756e-17, -2.77555756e-17,  5.00000000e-01],
        [ 3.33333333e-01,  3.33333333e-01,  8.33333333e-01],
        [ 6.66666667e-01,  6.66666667e-01,  1.66666667e-01],
        [-5.55111512e-17,  5.00000000e-01,  5.00000000e-01],
        [ 6.66666667e-01,  1.66666667e-01,  1.66666667e-01],
        [ 3.33333333e-01,  8.33333333e-01,  8.33333333e-01]]),
 array(['Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu',
        'Cu'], dtype='<U2'),
 array([[ 1.,  0., -1.],
        [ 1., -1.,  0.],
        [ 1.,  1.,  1.]]))

In [10]:
my_interface.atoms_1, my_interface.elements_1,my_interface.lattice_1 = super_cell(cell,np.eye(3),my_interface.atoms_1,my_interface.elements_1)

In [61]:
my_interface.lattice_1

array([[ 1.,  0., -1.],
       [ 1., -1.,  0.],
       [ 1.,  1.,  1.]])

In [11]:
my_interface.atoms_2, my_interface.elements_2,my_interface.lattice_2 = my_interface.atoms_1.copy(), my_interface.elements_1.copy(),my_interface.lattice_1.copy()

In [12]:
my_interface.lattice_2_TD = my_interface.lattice_1.copy()

In [110]:
# 自分が作ったLatticeでの軸、いまはｘ軸が(1,1,1)のLatticeを作って、その軸の周りに回転させようとしているから[1,0,0]
# つまりsuper cellでの回転軸は常に(1,0,0)
rotation_axis = [1,0,0]
my_interface.parse_limit(du = 1e-4, S  = 1e-4, sgm1=100, sgm2=100, dd = 1e-4)

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
my_interface.search_one_position(rotation_axis,38.94,1,0.01)

[1. 1. 0.]
rotation matrix = [[ 0.88890228  0.11109772  0.44442101]
 [ 0.11109772  0.88890228 -0.44442101]
 [-0.44442101  0.44442101  0.77780456]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  0]
 [ 0 -1  4]
 [ 0 -2 -1]]; sigma_1 = 9

U2 = 
[[-1  0  0]
 [ 0  1  4]
 [ 0 -2  1]]; sigma_2 = 9

D = 
[[ 1.000e+00  0.000e+00  3.013e-05]
 [ 0.000e+00  1.000e+00 -3.013e-05]
 [-3.013e-05  3.013e-05  1.000e+00]]

axis = [1. 1. 0.] ; theta = 38.94



In [104]:
axis = np.array([1,1,0])

In [111]:
# axis = np.array([1,1,1])
# hkl_perp_axis = MID(my_interface.CSL, np.array([1,1,1]))
hkl_perp_axis = MID(my_interface.CSL, axis)
a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T
# b = a+b
hkl_1 = MID(my_interface.lattice_1,a)
hkl_2 = MID(my_interface.lattice_1,b)

In [ ]:
# axis = np.array([1,1,1])
# hkl_perp_axis = MID(my_interface.CSL, np.array([1,1,1]))
hkl_perp_axis = MID(my_interface.CSL, np.array([1,1,1]))
a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T
# b = a+b
hkl_1 = MID(my_interface.lattice_1,a)
hkl_2 = MID(my_interface.lattice_1,b)

In [112]:
a

array([ 2., -2., -1.])

In [113]:
b

array([ 1., -1.,  4.])

In [275]:
print_list([1,1,1],100)

Sigma:     1  Theta:   0.00 
Sigma:     3  Theta:  60.00 
Sigma:     7  Theta:  38.21 
Sigma:    13  Theta:  27.80 
Sigma:    19  Theta:  46.83 
Sigma:    21  Theta:  21.79 
Sigma:    31  Theta:  17.90 
Sigma:    37  Theta:  50.57 
Sigma:    39  Theta:  32.20 
Sigma:    43  Theta:  15.18 
Sigma:    49  Theta:  43.57 
Sigma:    57  Theta:  13.17 
Sigma:    61  Theta:  52.66 
Sigma:    67  Theta:  24.43 
Sigma:    73  Theta:  11.64 
Sigma:    79  Theta:  33.99 
Sigma:    91  Theta:  10.42 
Sigma:    93  Theta:  42.10 
Sigma:    97  Theta:  30.59 


In [276]:
my_interface = core('Cu_mp-30_conventional_standard.cif','Cu_mp-30_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


In [277]:
axis = np.array([1,1,1])

In [278]:
# ここではconventional cellをmy_interface.lattice_1にして計算をしている
# axis = np.array([1,1,1])
a,b = get_pri_vec_inplane(axis, np.array(np.round(dot(inv(my_interface.lattice_1), my_interface.lattice_1),8),dtype = int)).T
cell = np.column_stack((axis,a,b))
my_interface.atoms_1, my_interface.elements_1,my_interface.lattice_1 = super_cell(cell,np.eye(3),my_interface.atoms_1,my_interface.elements_1)
my_interface.atoms_2, my_interface.elements_2,my_interface.lattice_2 = my_interface.atoms_1.copy(), my_interface.elements_1.copy(),my_interface.lattice_1.copy()
my_interface.lattice_2_TD = my_interface.lattice_1.copy()

In [279]:
# 自分が作ったLatticeでの軸、いまはｘ軸が(1,1,1)のLatticeを作って、その軸の周りに回転させようとしているから[1,0,0]
# つまりsuper cellでの回転軸は常に(1,0,0)
rotation_axis = [1,0,0]
my_interface.parse_limit(du = 1e-4, S  = 1e-4, sgm1=100, sgm2=100, dd = 1e-4)

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
my_interface.search_one_position(rotation_axis,27.80,1,0.01)

[1. 1. 1.]
rotation matrix = [[ 0.92305398 -0.23079544  0.30774146]
 [ 0.30774146  0.92305398 -0.23079544]
 [-0.23079544  0.30774146  0.92305398]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  0]
 [ 0  4  1]
 [ 0 -1  3]]; sigma_1 = 13

U2 = 
[[-1  0  0]
 [ 0  3 -1]
 [ 0  1  4]]; sigma_2 = 13

D = 
[[ 1.00e+00  4.26e-05 -4.26e-05]
 [-4.26e-05  1.00e+00  4.26e-05]
 [ 4.26e-05 -4.26e-05  1.00e+00]]

axis = [1. 1. 1.] ; theta = 27.8



In [280]:
# axis = np.array([1,1,1])
# hkl_perp_axis = MID(my_interface.CSL, np.array([1,1,1]))
hkl_perp_axis = MID(my_interface.CSL, axis)
a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T
# b = a+b
hkl_1 = MID(my_interface.lattice_1,a)
hkl_2 = MID(my_interface.lattice_1,b)

In [281]:
my_interface.compute_bicrystal(hkl_1, normal_ortho = True, plane_ortho = True, lim = 50, tol = 1e-2)

cell 1:
[[ 0 -1  0]
 [-4  0 -2]
 [ 1  0  7]]
cell 2:
[[ 0 -1  0]
 [-4  0 -2]
 [ 1  0  7]]


In [282]:
my_interface.get_bicrystal(xyz_1 = [2,2,2], xyz_2 = [2,2,2],filetype="LAMMPS",filename="atominfile")

[[ 0 -1  0]
 [-4  0 -2]
 [ 1  0  7]]
[[ 0 -1  0]
 [-3  0 -5]
 [-1  0  7]]
num of atoms1 = 312
num of atoms2 = 312
[[ 1.01980390e+01 -2.22044605e-16 -3.77475828e-15]
 [ 4.44089210e-16  3.46410162e+00 -2.66453526e-15]
 [ 2.22044605e-16  4.44089210e-16  1.76635217e+01]]


In [588]:
my_interface = core('Cu_mp-30_conventional_standard.cif','Cu_mp-30_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


In [614]:
my_interface = core("Fe_mp-13_conventional_standard.cif","Fe_mp-13_conventional_standard.cif")

Warning!, this programme will rewrite the POSCAR file in this dir!


In [615]:
axis = np.array([1,1,0])

In [616]:
print_list(axis,100)

Sigma:     1  Theta:   0.00 
Sigma:     3  Theta:  70.53 
Sigma:     9  Theta:  38.94 
Sigma:    11  Theta:  50.48 
Sigma:    17  Theta:  86.63 
Sigma:    19  Theta:  26.53 
Sigma:    27  Theta:  31.59 
Sigma:    33  Theta:  20.05 
Sigma:    41  Theta:  55.88 
Sigma:    43  Theta:  80.63 
Sigma:    51  Theta:  16.10 
Sigma:    57  Theta:  44.00 
Sigma:    59  Theta:  45.98 
Sigma:    67  Theta:  62.44 
Sigma:    73  Theta:  13.44 
Sigma:    81  Theta:  77.88 
Sigma:    83  Theta:  17.86 
Sigma:    89  Theta:  34.89 
Sigma:    97  Theta:  61.02 
Sigma:    99  Theta:  11.54 


In [617]:
# ここではconventional cellをmy_interface.lattice_1にして計算をしている
# axis = np.array([1,1,1])
a,b = get_pri_vec_inplane(axis, np.array(np.round(dot(inv(my_interface.lattice_1), my_interface.lattice_1),8),dtype = int)).T
cell = np.column_stack((axis,a,b))
my_interface.atoms_1, my_interface.elements_1,my_interface.lattice_1 = super_cell(cell,my_interface.lattice_1,my_interface.atoms_1,my_interface.elements_1)
my_interface.atoms_2, my_interface.elements_2,my_interface.lattice_2 = my_interface.atoms_1.copy(), my_interface.elements_1.copy(),my_interface.lattice_1.copy()
my_interface.lattice_2_TD = my_interface.lattice_1.copy()

In [618]:
# 自分が作ったLatticeでの軸、いまはｘ軸が(1,1,1)のLatticeを作って、その軸の周りに回転させようとしているから[1,0,0]
# つまりsuper cellでの回転軸は常に(1,0,0)
rotation_axis = [1,0,0]
my_interface.parse_limit(du = 1e-4, S  = 1e-4, sgm1=100, sgm2=100, dd = 1e-4)

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
my_interface.search_one_position(rotation_axis,38.94,1,0.01)

[2.84005168e+00 2.84005168e+00 3.47806020e-16]
rotation matrix = [[ 0.88890228  0.11109772  0.44442101]
 [ 0.11109772  0.88890228 -0.44442101]
 [-0.44442101  0.44442101  0.77780456]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  0]
 [ 0 -1  4]
 [ 0 -2 -1]]; sigma_1 = 9

U2 = 
[[-1  0  0]
 [ 0  1  4]
 [ 0 -2  1]]; sigma_2 = 9

D = 
[[ 1.000e+00  0.000e+00  3.013e-05]
 [ 0.000e+00  1.000e+00 -3.013e-05]
 [-3.013e-05  3.013e-05  1.000e+00]]

axis = [2.84005168e+00 2.84005168e+00 3.47806020e-16] ; theta = 38.94



In [619]:
# axis = np.array([1,1,1])
# hkl_perp_axis = MID(my_interface.CSL, np.array([1,1,1]))
hkl_perp_axis = MID(my_interface.CSL, axis)
a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T
# b = a+b
hkl_1 = MID(my_interface.lattice_1,a)
hkl_2 = MID(my_interface.lattice_1,b)

In [620]:
my_interface.compute_bicrystal(hkl_1, normal_ortho = True, plane_ortho = True, lim = 50, tol = 1e-2)

cell 1:
[[ 0 -1  0]
 [ 1  0  4]
 [ 2  0 -1]]
cell 2:
[[ 0 -1  0]
 [ 1  0  4]
 [ 2  0 -1]]


In [621]:
my_interface.get_bicrystal(xyz_1 = [2,1,1], xyz_2 = [2,1,1],filetype="LAMMPS",filename="atominfile")

[[ 0 -1  0]
 [ 1  0  4]
 [ 2  0 -1]]
[[ 0 -1  0]
 [-1  0  4]
 [ 2  0  1]]
num of atoms1 = 36
num of atoms2 = 36
[[ 1.70403101e+01 -5.28936393e-16  1.95383952e-16]
 [-7.12597445e-16  4.01643960e+00 -7.67777869e-16]
 [-5.60180554e-16  2.83941496e-17  1.20493188e+01]]


In [624]:
105.54022915+0.31041244

105.85064159

In [625]:
105.85064159+1e-5

105.85065159

In [627]:
217.98292197/2+0.21040823

109.201869215

In [628]:
109.201869215+1e-5

109.201879215

In [641]:
def read_atomin_id(filename):
    #read atom positions from POSCAR
    with open (f"{filename}",'r') as f:
        lines = f.readlines()
    atoms = np.array([0,0,0,0,0])
    for i in range(14,len(lines)):
        atoms = np.vstack((atoms,np.array(lines[i].split(),dtype = float)))
    atoms = np.delete(atoms,0,axis= 0)
    atoms = np.delete(atoms,[0],axis=1)
    a = float(lines[6].split()[1])
    b = float(lines[7].split()[1])
    c = float(lines[8].split()[1])
    return a, b, c, atoms

In [711]:
dirname_list = ["9_7_0","11_9_0","12_5_0"]

In [712]:
for dirname in dirname_list:
    a, b, c, atoms = read_atomin_id(f"{dirname}/atominfile3")
    add_atoms = atoms[atoms[:,1]==np.sort(np.unique(atoms[:,1]))[1]]
    add_atoms[:,1] = add_atoms[:,1] + a/2
    add_atoms[:,0] = 2
    atoms_tot = np.vstack((atoms,add_atoms))
    Write_to_lammps(a,b,c,atoms_tot,f"{dirname}/atominfile4")
    new_cell_boundary = np.unique(np.round(add_atoms[:,1],5))[0]
    tol = 1e-5
    new_cell_boundary = new_cell_boundary + tol
    rewrite_blockfile(new_cell_boundary,dirname)

In [722]:
dirname = "11_6_0"
a, b, c, atoms = read_atomin_id(f"{dirname}/atominfile3")

In [723]:
add_atoms = atoms[atoms[:,1]==np.sort(np.unique(atoms[:,1]))[1]]
add_atoms[:,1] = add_atoms[:,1] + a/2

In [724]:
add_atoms[:,0] = 2

In [725]:
atoms_tot = np.vstack((atoms,add_atoms))

In [726]:
Write_to_lammps(a,b,c,atoms_tot,f"{dirname}/atominfile4")

In [727]:
new_cell_boundary = np.unique(np.round(add_atoms[:,1],5))[0]

In [728]:
tol = 1e-5

In [729]:
new_cell_boundary = new_cell_boundary + tol

In [730]:
rewrite_blockfile(new_cell_boundary,dirname)

In [693]:
def replace_func(dirname,replace_set):
    target, replace = replace_set
    
    with open(f"{dirname}/blockfile", 'r') as f1:
        tmp_list =[]
        for row in f1:
            if row.find(target) != -1:
                tmp_list.append(replace)
            else:
                tmp_list.append(row)
    
    with open(f"{dirname}/blockfile2", 'w') as f2:
        for i in range(len(tmp_list)):
            f2.write(tmp_list[i])

In [694]:
def rewrite_blockfile(new_cell_boundary,dirname):
    proto2_file_path = os.path.join(os.getcwd(),dirname)
    replace_setA = ('region right block', f'region right block {new_cell_boundary} EDGE EDGE EDGE EDGE EDGE units box\n') 
    replace_func(dirname,replace_setA)

In [700]:
def Write_to_lammps(a,b,c,supercell_atoms,filename):
    dim = np.array([1,1,1])
    X = supercell_atoms.copy()

    NumberAt = len(X) 

    dimx, dimy, dimz = dim

    xlo = 0.00000000
    xhi = a
    ylo = 0.00000000
    yhi = b 
    zlo = 0.00000000
    zhi = c

    yz = 0.0

    Counter = np.arange(1, NumberAt + 1).reshape(1, -1)

    # data = np.concatenate((X_new, Y_new))
    FinalMat = np.concatenate((Counter.T, X), axis=1)

    with open(filename, 'w') as f:
        f.write('#Header \n \n')
        f.write('{} atoms \n \n'.format(NumberAt))
        f.write('2 atom types \n \n')
        f.write('{0:.8f} {1:.8f} xlo xhi \n'.format(xlo, xhi))
        f.write('{0:.8f} {1:.8f} ylo yhi \n'.format(ylo, yhi))
        f.write('{0:.8f} {1:.8f} zlo zhi \n\n'.format(zlo, zhi))
        f.write('{0:.8f} {1:.8f} {2:.8f} xy xz yz \n\n'.format(0, 0, yz))            
        f.write('Atoms \n \n')
        np.savetxt(f, FinalMat, fmt='%i %i %.8f %.8f %.8f')
    f.close()

In [7]:
# CSLとaxisのlattice_1の直行座標系
def get_a_b(CSL, axis):
    hkl_perp_axis = MID(CSL, axis)
    a, b = get_pri_vec_inplane(hkl_perp_axis, CSL).T

    if(norm(cross(axis,[1,0,0])) < 1e-8):
        b = a + b
    elif (norm(cross(axis,[1,1,1])) < 1e-8):
        if dot(a,b) < 0:
            b = a + b
        b = a + b
    if (abs(norm(a) - norm(b)) < 1e-8):
        raise RuntimeError ('the tow vectors are identical!')

    return a.T, b.T

symmetrical tilt のときのhklの計算の仕方

In [85]:
hkl_1

array([0, 7, 2])

In [86]:
hkl_2

array([0, 4, 3])

cell 1:
[[ 0 -1  0]
 [ 3  0  1]
 [-1  0 -5]]
cell 2:
[[ 0 -1  0]
 [ 3  0  1]
 [-1  0 -5]]


[[ 0 -1  0]
 [ 3  0  1]
 [-1  0 -5]]
[[ 0 -1  0]
 [ 2  0  4]
 [ 1  0 -5]]
num of atoms1 = 168
num of atoms2 = 168
[[ 7.48331477e+00 -4.44089210e-16 -8.88178420e-16]
 [ 0.00000000e+00  3.46410162e+00 -8.88178420e-16]
 [ 0.00000000e+00  2.22044605e-16  1.29614814e+01]]


In [ ]:
def overwrite_supercell(interface,axis_input,n):
    """
    axis_input: 自分が入れる回転軸のpremitive cellでの表示
    n: 粒界面を指定するベクトルの直交座標表示
    """
    axis_cart = dot(interface.lattice_1, axis_input)
    axis_int = np.array(np.round(dot(inv(interface.conv_lattice_1), axis_cart),8),dtype = int)

    n_1_int = np.array(np.round(dot(inv(interface.conv_lattice_1), n),8),dtype = int)
    n_1_pre = np.array(np.round(dot(inv(interface.lattice_1),n),8),dtype = int)
    v3 = cross(n,axis_int)
    #print(interface.lattice_1)
    v3_int = np.array(np.round(dot(inv(interface.lattice_1), v3),8),dtype = int)
    v3_int_reduced = v3_int/abs(np.gcd.reduce(v3_int))
    bicrystal_U1 = np.column_stack([n_1_pre,axis_input])
    bicrystal_U1 = np.column_stack([bicrystal_U1,v3_int_reduced])
    interface.bicrystal_U1 = bicrystal_U1
    supercell = dot(interface.lattice_1,bicrystal_U1)

    bicrystal_U2 = np.array(np.round(dot(inv(interface.lattice_2_TD), supercell),8),dtype = int)
    interface.bicrystal_U2 = bicrystal_U2

In [ ]:
def get_gb_files(interface, hkl, axis, sigma, axis_name, hkl_name, ab, axis_num, ab_num, bond_length,n,surface,name,element,stgb):
    interface.compute_bicrystal(hkl, normal_ortho = True, plane_ortho = True, lim = 50, tol = 1e-10)
#    print(f'U1 is {interface.bicrystal_U1}')
    half_cell = dot(interface.lattice_1, interface.bicrystal_U1)
#    print(f'lattice1_2 is {interface.lattice_1}')
    x,y,z = get_expansion_xyz(half_cell)
    axis_x, axis_y, axis_z = axis_name
    axis_name_num = 100*axis_x + 10*axis_y +axis_z
    hkl_name = np.array(hkl_name, dtype=int)
    hkl_name_num = hkl_name.copy()
    hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
    hkl_x, hkl_y, hkl_z = hkl_name
    dirname = f"{str(hkl_x)}_{str(hkl_y)}_{str(hkl_z)}_{hkl_name_num[0]}_{hkl_name_num[1]}_{hkl_name_num[2]}_gb"

    if stgb==False:
        dirname = f"{str(hkl_x)}_{str(hkl_y)}_{str(hkl_z)}"
        print("====================================================")
    else:
        pass
    os.makedirs(dirname,exist_ok=True)
    os.chdir(dirname)

    overwrite_supercell(interface,axis,n)

    
    if (axis_name == [1,1,1]):    
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = True)
    else:
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = False)
    
    CNID_hkl = dot(inv(interface.conv_lattice_1),interface.CNID)

    
    eps = 1e-5
    define_bicrystal_regions(interface.xhi)

    CNID = dot(interface.orient, interface.CNID)  
    
    v1 = np.array([0,1.,0])*CNID[:,0][1] + np.array([0,0,1.])*CNID[:,0][2]
    v2 = np.array([0,1.,0])*abs(CNID[:,1][1]) + np.array([0,0,1.])*abs(CNID[:,1][2])
    v2 = -v2


    
    n1 = int(ceil(norm(v1)/0.2))
    n2 = int(ceil(norm(v2)/0.2))
    write_trans_file(v1,v2,n1,n2)

    with open(f"{name}.meam","w") as f:
        f.writelines(element.meam())

    with open("library.meam","w") as f:
        f.writelines(element.library_meam())

    if stgb==False:
        with open("proto.in","w") as f:
            f.writelines(element.proto())
    else:
        pass

    os.chdir(os.pardir)

In [ ]:
def define_bicrystal_regions(xhi):
    tol = 1e-5
    """
    generate a file defining some regions in the LAMMPS and define the atoms
    inside these regions into some groups.
    argument:
    region_names --- list of name of regions
    region_los --- list of the low bounds
    region_his --- list of the hi bounds
    """
    end_fixbulk1 = xhi/2-30
    start_fixbulk2 = xhi/2+30
    start_middle = xhi/2-20
    end_middle = xhi/2+20
    start_right = xhi/2 + tol
    start_bulk = 160
    end_bulk = 165


    with open('blockfile', 'w') as fb:
        fb.write('region fixbulk1 block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(end_fixbulk1))
        fb.write('region fixbulk2 block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
        format(start_fixbulk2))
        fb.write('region middle block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(start_middle,end_middle))
        fb.write('region right block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
        format(start_right))
        fb.write('region bulk block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(start_bulk,end_bulk))
        fb.write('group fixbulk1 region fixbulk1 \n')
        fb.write('group fixbulk2 region fixbulk2 \n')
        fb.write('group middle region middle \n')
        fb.write('group right region right \n')
        fb.write('group bulk region bulk \n')

In [ ]:
def get_all_STGBs(axis_list, theta_list, sigma_list,name,element,max_sigma,stgb):
    eps = 1e-5
    cif_filename = glob.glob(f"C:/Users/hatay/calc/python_script/{name}/*.cif")[0]
    my_interface = core(cif_filename, cif_filename)
    my_interface.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd =  1e-4)
    factor = element.getLatP()/(2*norm(my_interface.lattice_1[0,0]))
    my_interface.lattice_1 =  my_interface.lattice_1*factor
    my_interface.lattice_2 =  my_interface.lattice_2*factor
    my_interface.conv_lattice_1 =  my_interface.conv_lattice_1*factor
    my_interface.conv_lattice_2 =  my_interface.conv_lattice_2*factor  
    count = 1

    LatP = element.getLatP()
    bond_length = LatP/np.sqrt(2)
    coodinates = 12
    cut_off = 1.05    
    surface = Surface(LatP,bond_length,coodinates,cut_off)
    
    for i in range(len(axis_list)):
        axis = axis_list[i]
        axis_name = axis_list[i]
        axis_cart = dot(my_interface.conv_lattice_1, axis)

        axis = np.array(np.round(dot(inv(my_interface.lattice_1), axis_cart),8),dtype = int)
        for j in range(len(sigma_list[i])):

            sigma = sigma_list[i][j]
            my_interface.search_one_position(axis,theta_list[i][j]-0.001,1,0.001)
            CSL = my_interface.CSL
            n_1, n_2 = get_a_b(CSL, dot(my_interface.lattice_1,axis))

            hkl_1, hkl_2 = get_STGB_MLs(my_interface.lattice_1, n_1, n_2)

            hkl_name_1 = get_primitive_hkl(hkl_1, my_interface.lattice_1, my_interface.conv_lattice_1)
            hkl_name_2 = get_primitive_hkl(hkl_2, my_interface.lattice_2, my_interface.conv_lattice_2)
            
            get_gb_files(my_interface, hkl_1, axis, sigma, axis_name, hkl_name_1 ,n_1 ,name,element,stgb)

            get_gb_files(my_interface, hkl_2, axis, sigma, axis_name, hkl_name_2, n_2 ,name,element,stgb)


In [ ]:
def get_theta_sigma_list(axis_list,max_sigma):
    theta_list = []
    sigma_list = []
    for i in axis_list:
        lists, thetas = getsigmas(i, max_sigma)
        theta_list.append(thetas)
        sigma_list.append(lists)
    return theta_list,sigma_list